<span style="color:gray">
Fraunhofer Institute for Integrated Circuits IIS, Division Engineering of Adaptive Systems EAS<br>
Münchner Straße 16, 01187 Dresden, Germany
</span>

---

## ESB - Energy Saving by Blockchain

---

## Detection of Electric Vehicles and Photovoltaic Systems in Smart Meter Data

---

# 2: Data Preprocessing EV


In these notebook, the time series from the locations New York, California, and Austin locations are split into training, testing, and validation data of different input time spans. Furthermore, a kernel density estimate is applied to the split datasets.

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import train_test_split

In [ ]:
path=''

newyork = pd.read_csv (path + '1hour_data_newyork.csv', header=0, sep=',',decimal=".")
california = pd.read_csv (path + '1hour_data_california.csv', header=0, sep=',',decimal=".")
austin = pd.read_csv (path + '1hour_data_austin.csv', header=0, sep=',',decimal=".")


california_df = pd.read_csv (path+'1hour_data_california.csv',header=0, sep=',',decimal=".")
california_df.loc[:,"local_15min"] = pd.to_datetime(california_df.loc[:,"local_15min"])
california_df.loc[:,"year_month"] = california_df["local_15min"].dt.year.astype("str")\
    + "_" + california_df["local_15min"].dt.month.apply(lambda x: f"{x:02d}")

austin_df = pd.read_csv (path+'1hour_data_austin.csv',header=0, sep=',',decimal=".")
austin_df.loc[:,"local_15min"] = pd.to_datetime(austin_df.loc[:,"local_15min"])
austin_df.loc[:,"year_month"] = austin_df["local_15min"].dt.year.astype("str")\
    + "_" + austin_df["local_15min"].dt.month.apply(lambda x: f"{x:02d}")

newyork_df = pd.read_csv (path+'1hour_data_newyork.csv',header=0, sep=',',decimal=".")
newyork_df.loc[:,"local_15min"] = pd.to_datetime(newyork_df.loc[:,"local_15min"])
newyork_df.loc[:,"year_month"] = newyork_df["local_15min"].dt.year.astype("str")\
    + "_" + newyork_df["local_15min"].dt.month.apply(lambda x: f"{x:02d}")

In [ ]:
california_customer_has_ev = california_df.groupby("dataid")[["car1", "car2"]].max().max(axis=1)>0
california_customers_with_ev = list(california_customer_has_ev[california_customer_has_ev].index)
california_customers_without_ev = list(california_customer_has_ev[~california_customer_has_ev].index)

california_df_with_EV = california_df.loc[california_df["dataid"].isin(california_customers_with_ev),:]
california_df_without_EV = california_df.loc[california_df["dataid"].isin(california_customers_without_ev),:]

print("California:",
      f"\nThe customers {california_customers_with_ev} have an EV.",
      f"\nThe customers {california_customers_without_ev} do not have an EV.")


austin_customer_has_ev = austin_df.groupby("dataid")[["car1", "car2"]].max().max(axis=1)>0
austin_customers_with_ev = list(austin_customer_has_ev[austin_customer_has_ev].index)
austin_customers_without_ev = list(austin_customer_has_ev[~austin_customer_has_ev].index)

austin_df_with_EV = austin_df.loc[austin_df["dataid"].isin(austin_customers_with_ev),:]
austin_df_without_EV = austin_df.loc[austin_df["dataid"].isin(austin_customers_without_ev),:]

print("austin:",
      f"\nThe customers {austin_customers_with_ev} have an EV.",
      f"\nThe customers {austin_customers_without_ev} do not have an EV.")


newyork_customer_has_ev = newyork_df.groupby("dataid")[["car1", "car2"]].max().max(axis=1)>0
newyork_customers_with_ev = list(newyork_customer_has_ev[newyork_customer_has_ev].index)
newyork_customers_without_ev = list(newyork_customer_has_ev[~newyork_customer_has_ev].index)

newyork_df_with_EV = newyork_df.loc[newyork_df["dataid"].isin(newyork_customers_with_ev),:]
newyork_df_without_EV = newyork_df.loc[newyork_df["dataid"].isin(newyork_customers_without_ev),:]

print("New York:",
      f"\nThe customers {newyork_customers_with_ev} have an EV.",
      f"\nThe customers {newyork_customers_without_ev} do not have an EV.")

In [ ]:
newyork_df_with_EV.loc[:,"customer"] = newyork_df_with_EV.loc[:,"dataid"].apply(lambda x: f"ny_{x}")
newyork_df_without_EV.loc[:,"customer"] = newyork_df_without_EV.loc[:,"dataid"].apply(lambda x: f"ny_{x}")
austin_df_with_EV.loc[:,"customer"] = austin_df_with_EV.loc[:,"dataid"].apply(lambda x: f"a_{x}")
austin_df_without_EV.loc[:,"customer"] = austin_df_without_EV.loc[:,"dataid"].apply(lambda x: f"a_{x}")
california_df_with_EV.loc[:,"customer"] = california_df_with_EV.loc[:,"dataid"].apply(lambda x: f"c_{x}")
california_df_without_EV.loc[:,"customer"] = california_df_without_EV.loc[:,"dataid"].apply(lambda x: f"c_{x}")

In [ ]:
with_ev_df = pd.concat([newyork_df_with_EV, california_df_with_EV, austin_df_with_EV]).drop(columns="dataid")
without_ev_df = pd.concat([newyork_df_without_EV, california_df_without_EV, austin_df_without_EV]).drop(columns="dataid")

#### Split the dataset into 80% training data and 20% validation data

In [ ]:
ts = 0.8
customers_with_ev_train, customers_with_ev_val = train_test_split(with_ev_df["customer"].unique(),
                                                                  train_size=ts, random_state=123)
customers_without_ev_train, customers_without_ev_val = train_test_split(without_ev_df["customer"].unique(),
                                                                        train_size=ts, random_state=123)

#### Households from the synthetic dataset are used only in the training and test dataset

In [ ]:
def swap_synthetic_data(customers_train, customers_val):
    
    def swap(A,B,i,j):
        temp = B[j]
        B[j] = A[i]
        A[i] = temp
        return A,B
    
    random.seed(111)

    for i in range(len(customers_val)):
        if customers_val[i][0] == 'c':
            print(i)
            while True:
                num = random.randint(0, len(customers_train)-1)
                if customers_train[num][0] != 'c' and customers_train[num] not in customers_val:
                    swap(customers_val,customers_train,i,num)
                    break

In [ ]:
swap_synthetic_data(customers_with_ev_train, customers_with_ev_val)
swap_synthetic_data(customers_without_ev_train, customers_without_ev_val)

#### Split training data into 70% training data and 10% test data

In [ ]:
ts = 0.9
customers_with_ev_train, customers_with_ev_test = train_test_split(customers_with_ev_train,
                                                                  train_size=ts, random_state=123)
customers_without_ev_train, customers_without_ev_test = train_test_split(customers_without_ev_train,
                                                                        train_size=ts, random_state=123)

### Monthly input time span

In [ ]:
with_ev_df['Month_Number'] = with_ev_df['local_15min'].dt.month

grouped_with_ev = with_ev_df.groupby(['customer', 'Month_Number']).agg({'Month_Number': 'first'})
grouped_with_ev=grouped_with_ev.reset_index('customer')
grouped_with_ev.reset_index(drop=True, inplace=True)
grouped_with_ev=grouped_with_ev.to_numpy()


without_ev_df['Month_Number'] = without_ev_df['local_15min'].dt.month

grouped_without_ev = without_ev_df.groupby(['customer', 'Month_Number']).agg({'Month_Number': 'first'})
grouped_without_ev=grouped_without_ev.reset_index('customer')
grouped_without_ev.reset_index(drop=True, inplace=True)
grouped_without_ev=grouped_without_ev.to_numpy()

In [ ]:
def create_train_test_val_set(customers_with_ev, customers_without_ev, time_interval):
    
    x_set = pd.DataFrame()
    for i in customers_with_ev:
        print(i + "-----------------------------------")
        for j in grouped_with_ev:
            if i == j[0]:
                print(j)
                x_set=pd.concat([x_set,with_ev_df['grid'].loc[(with_ev_df['customer'] == j[0]) & (with_ev_df[time_interval] == j[1])].reset_index(drop=True)], ignore_index=True, axis=1)

    y_set = np.ones(x_set.shape[1])
    
    for i in customers_without_ev:
        print(i + "-----------------------------------")
        for j in grouped_without_ev:
            if i == j[0]:
                print(j)
                x_set=pd.concat([x_set,without_ev_df['grid'].loc[(without_ev_df['customer'] == j[0]) & (without_ev_df[time_interval] == j[1])].reset_index(drop=True)], ignore_index=True, axis=1)

    y_set = pd.DataFrame(np.concatenate([y_set, np.zeros(x_set.shape[1] - len(y_set))])).astype(int)
    
    if time_interval == 'Month_Number':
        x_set=x_set.loc[:719,:].T.fillna(0)
        
    if time_interval == 'Week_Number':
        x_set=x_set.loc[:167,:].T.fillna(0)
        
    if time_interval == 'Day_Number':
        x_set=x_set.T.fillna(0)
        
    return x_set, y_set

In [ ]:
X_train,Y_train=create_train_test_val_set(customers_with_ev_train, customers_without_ev_train, 'Month_Number')

X_test,Y_test=create_train_test_val_set(customers_with_ev_test, customers_without_ev_test, 'Month_Number')

X_val,Y_val=create_train_test_val_set(customers_with_ev_val, customers_without_ev_val, 'Month_Number')

In [ ]:
discr = np.linspace(-10, 20, 100).reshape(-1, 1)

def calc_kde(X, discr=discr):
    kde = KernelDensity(kernel='gaussian', bandwidth=.75).fit(X.values.reshape(-1, 1))
    return np.exp(kde.score_samples(discr))

In [ ]:
X_train_KDE = pd.DataFrame(np.stack(X_train.apply(calc_kde, axis=1).values))

X_test_KDE = pd.DataFrame(np.stack(X_test.apply(calc_kde, axis=1).values))

X_val_KDE = pd.DataFrame(np.stack(X_val.apply(calc_kde, axis=1).values))

In [ ]:
path = ''

X_train.to_csv(path + 'ev/monthly/X_train.csv', sep=',', encoding='utf-8', index=False)
X_test.to_csv(path + 'ev/monthly/X_test.csv', sep=',', encoding='utf-8', index=False)
X_val.to_csv(path + 'ev/monthly/X_val.csv', sep=',', encoding='utf-8', index=False)

X_train_KDE.to_csv(path + 'ev/monthly/X_train_KDE.csv', sep=',', encoding='utf-8', index=False)
X_test_KDE.to_csv(path + 'ev/monthly/X_test_KDE.csv', sep=',', encoding='utf-8', index=False)
X_val_KDE.to_csv(path + 'ev/monthly/X_val_KDE.csv', sep=',', encoding='utf-8', index=False)

Y_train.to_csv(path + 'ev/monthly/Y_train.csv', sep=',', encoding='utf-8', index=False)
Y_test.to_csv(path + 'ev/monthly/Y_test.csv', sep=',', encoding='utf-8', index=False)
Y_val.to_csv(path + 'ev/monthly/Y_val.csv', sep=',', encoding='utf-8', index=False)

### Weekly input time span

In [ ]:
with_ev_df['Week_Number'] = with_ev_df['local_15min'].dt.week

grouped_with_ev = with_ev_df.groupby(['customer', 'Week_Number']).agg({'Week_Number': 'first'})
grouped_with_ev=grouped_with_ev.reset_index('customer')
grouped_with_ev.reset_index(drop=True, inplace=True)
grouped_with_ev=grouped_with_ev.to_numpy()


without_ev_df['Week_Number'] = without_ev_df['local_15min'].dt.week

grouped_without_ev = without_ev_df.groupby(['customer', 'Week_Number']).agg({'Week_Number': 'first'})
grouped_without_ev=grouped_without_ev.reset_index('customer')
grouped_without_ev.reset_index(drop=True, inplace=True)
grouped_without_ev=grouped_without_ev.to_numpy()

In [ ]:
X_train,Y_train=create_train_test_val_set(customers_with_ev_train, customers_without_ev_train, 'Week_Number')

X_test,Y_test=create_train_test_val_set(customers_with_ev_test, customers_without_ev_test, 'Week_Number')

X_val,Y_val=create_train_test_val_set(customers_with_ev_val, customers_without_ev_val, 'Week_Number')

In [ ]:
X_train_KDE = pd.DataFrame(np.stack(X_train.apply(calc_kde, axis=1).values))

X_test_KDE = pd.DataFrame(np.stack(X_test.apply(calc_kde, axis=1).values))

X_val_KDE = pd.DataFrame(np.stack(X_val.apply(calc_kde, axis=1).values))

In [ ]:
path = ''

X_train.to_csv(path + 'ev/weekly/X_train.csv', sep=',', encoding='utf-8', index=False)
X_test.to_csv(path + 'ev/weekly/X_test.csv', sep=',', encoding='utf-8', index=False)
X_val.to_csv(path + 'ev/weekly/X_val.csv', sep=',', encoding='utf-8', index=False)

X_train_KDE.to_csv(path + 'ev/weekly/X_train_KDE.csv', sep=',', encoding='utf-8', index=False)
X_test_KDE.to_csv(path + 'ev/weekly/X_test_KDE.csv', sep=',', encoding='utf-8', index=False)
X_val_KDE.to_csv(path + 'ev/weekly/X_val_KDE.csv', sep=',', encoding='utf-8', index=False)

Y_train.to_csv(path + 'ev/weekly/Y_train.csv', sep=',', encoding='utf-8', index=False)
Y_test.to_csv(path + 'ev/weekly/Y_test.csv', sep=',', encoding='utf-8', index=False)
Y_val.to_csv(path + 'ev/weekly/Y_val.csv', sep=',', encoding='utf-8', index=False)

### Daily input time span

In [ ]:
with_ev_df['Day_Number'] = with_ev_df['local_15min'].dt.date

grouped_with_ev = with_ev_df.groupby(['customer', 'Day_Number']).agg({'Day_Number': 'first'})
grouped_with_ev=grouped_with_ev.reset_index('customer')
grouped_with_ev.reset_index(drop=True, inplace=True)
grouped_with_ev=grouped_with_ev.to_numpy()


without_ev_df['Day_Number'] = without_ev_df['local_15min'].dt.date

grouped_without_ev = without_ev_df.groupby(['customer', 'Day_Number']).agg({'Day_Number': 'first'})
grouped_without_ev=grouped_without_ev.reset_index('customer')
grouped_without_ev.reset_index(drop=True, inplace=True)
grouped_without_ev=grouped_without_ev.to_numpy()

In [ ]:
X_train,Y_train=create_train_test_val_set(customers_with_ev_train, customers_without_ev_train, 'Day_Number')

X_test,Y_test=create_train_test_val_set(customers_with_ev_test, customers_without_ev_test, 'Day_Number')

X_val,Y_val=create_train_test_val_set(customers_with_ev_val, customers_without_ev_val, 'Day_Number')

In [ ]:
X_train_KDE = pd.DataFrame(np.stack(X_train.apply(calc_kde, axis=1).values))

X_test_KDE = pd.DataFrame(np.stack(X_test.apply(calc_kde, axis=1).values))

X_val_KDE = pd.DataFrame(np.stack(X_val.apply(calc_kde, axis=1).values))

In [ ]:
path = ''

X_train.to_csv(path + 'ev/daily/X_train.csv', sep=',', encoding='utf-8', index=False)
X_test.to_csv(path + 'ev/daily/X_test.csv', sep=',', encoding='utf-8', index=False)
X_val.to_csv(path + 'ev/daily/X_val.csv', sep=',', encoding='utf-8', index=False)

X_train_KDE.to_csv(path + 'ev/daily/X_train_KDE.csv', sep=',', encoding='utf-8', index=False)
X_test_KDE.to_csv(path + 'ev/daily/X_test_KDE.csv', sep=',', encoding='utf-8', index=False)
X_val_KDE.to_csv(path + 'ev/daily/X_val_KDE.csv', sep=',', encoding='utf-8', index=False)

Y_train.to_csv(path + 'ev/daily/Y_train.csv', sep=',', encoding='utf-8', index=False)
Y_test.to_csv(path + 'ev/daily/Y_test.csv', sep=',', encoding='utf-8', index=False)
Y_val.to_csv(path + 'ev/daily/Y_val.csv', sep=',', encoding='utf-8', index=False)